In [2]:
import importlib
from __imports__ import *
import data, LSTM_returns, optimization

SOL_df = data.load_asset('SOLUSDT')
SOL_df = data.subset(SOL_df, start=pd.Timestamp('2021-01-01 08:00:00'), end=pd.Timestamp('2026-01-01'))
data.report_and_print_gaps(SOL_df)

SOL_df = data.add_returns(SOL_df)
print(SOL_df.columns)


Gap of 0 days 02:00:00 		 from 2021-02-11 05:00:00 to 2021-03-06 03:30:00
Gap of 0 days 03:00:00 		 from 2021-03-06 03:30:00 to 2021-04-20 04:30:00
Gap of 0 days 04:30:00 		 from 2021-04-20 04:30:00 to 2021-04-25 08:30:00
Gap of 0 days 05:00:00 		 from 2021-04-25 08:30:00 to 2021-08-13 06:30:00
Gap of 0 days 02:30:00 		 from 2021-08-13 06:30:00 to 2021-09-29 09:00:00
Gap of 0 days 01:30:00 		 from 2021-09-29 09:00:00 to 2023-03-24 14:00:00
Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Quote asset volume',
       'Number of trades', 'Taker buy base asset volume',
       'Taker buy quote asset volume', 'Return', 'Return_Target'],
      dtype='object')


In [4]:
# normalize all features but not targets
Fbutnt_df = SOL_df.copy()

numerical_columns = categorical_columns = "all"
exclude_columns = ['Return_Target']
Fbutnt_features = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume',
       'Taker buy quote asset volume', 'Return']
Fbutnt_df, Fbutnt_columns = data.normalize_data(Fbutnt_df, exclude_columns=exclude_columns)

Fbutnt_df = data.train_test_split(Fbutnt_df)
Fbutnt_columns

{'Open': MinMaxScaler(),
 'High': MinMaxScaler(),
 'Low': MinMaxScaler(),
 'Close': MinMaxScaler(),
 'Volume': MinMaxScaler(),
 'Quote asset volume': MinMaxScaler(),
 'Number of trades': MinMaxScaler(),
 'Taker buy base asset volume': MinMaxScaler(),
 'Taker buy quote asset volume': MinMaxScaler(),
 'Return': MinMaxScaler(),
 'Return_Target': 'standard'}

In [ ]:
import optuna

results = None
def objective(trial):
    global results
    # Example of hyperparameter suggestions
    hyperparams = {
        'lstm_units': trial.suggest_int('lstm_units', 10, 100),
        'dropout_rate': trial.suggest_float('dropout_rate', 0.0, 0.5),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
        'activation': trial.suggest_categorical('activation', ['relu', 'tanh', 'sigmoid']),
    }
    episode_lengths = trial.suggest_int('episode_length', 5, 80)
    hyperparams['batch_size'] = 128
    hyperparams['epochs'] = 1  # Set to 1 for quick testing; increase for actual training

    
    # Placeholder for model training and evaluation
    train_X_y = LSTM_returns.vectorize_train_data(Fbutnt_df, features=Fbutnt_features, episode_length=episode_lengths)
    hyperparams['input_shape'] = train_X_y['train_X'].shape[1:]
    
    LSTM = LSTM_returns.create(hyperparams)
    results = LSTM_returns.train(LSTM, hyperparams, train_X_y, evaluateAtEnd=False)
    return results.history['loss'][-1]  # Return the last loss value as the score


# Example Optuna study
study = optuna.create_study(direction='minimize')  # or 'maximize' depending on the objective
study.optimize(objective, n_trials=1)

# Print the best hyperparameters
print("Best hyperparameters:", study.best_params)

[I 2025-04-25 12:26:30,070] A new study created in memory with name: no-name-c6f00b1f-2df6-4180-8c35-bfab4d2559b4
C:\Users\ic2594\AppData\Local\Temp\ipykernel_11432\3775519433.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
c:\Users\ic2594\AppData\Local\miniconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


465/465 ━━━━━━━━━━━━━━━━━━━━ 11s 19ms/step - loss: 5.9088e-04 - mae: 0.0180


[I 2025-04-25 12:26:40,851] Trial 0 finished with value: 0.0 and parameters: {'lstm_units': 18, 'dropout_rate': 0.32577781396984473, 'learning_rate': 0.007201065899295995, 'activation': 'sigmoid', 'episode_length': 37}. Best is trial 0 with value: 0.0.


Best hyperparameters: {'lstm_units': 18, 'dropout_rate': 0.32577781396984473, 'learning_rate': 0.007201065899295995, 'activation': 'sigmoid', 'episode_length': 37}


In [27]:
import optuna.visualization as vis
import plotly

# Visualize parameter importance
param_importance_fig = vis.plot_param_importances(study)
param_importance_fig.show()

# Visualize parameter sensitivity
param_sensitivity_fig = vis.plot_slice(study)
param_sensitivity_fig.show()

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [14]:
results.history['loss'][-1], results.history['mae'][-1]

(0.0004368710797280073, 0.016942961141467094)